In [8]:
import pandas as pd
from pyproj import Geod

In [9]:
df=pd.read_csv('data/finalTrain.csv')

In [10]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [11]:
##handling null values
df.isnull().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [12]:
##droping id columns
df=df.drop(labels=['ID','Delivery_person_ID'],axis=1)

In [13]:
##droping latitude and longitude columns , Type_of_order
df.drop(labels=['Type_of_order','Order_Date','Time_Orderd','Time_Order_picked','Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude'],axis=1,inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Delivery_person_Age      43730 non-null  float64
 1   Delivery_person_Ratings  43676 non-null  float64
 2   Weather_conditions       44968 non-null  object 
 3   Road_traffic_density     44983 non-null  object 
 4   Vehicle_condition        45584 non-null  int64  
 5   Type_of_vehicle          45584 non-null  object 
 6   multiple_deliveries      44591 non-null  float64
 7   Festival                 45356 non-null  object 
 8   City                     44384 non-null  object 
 9   Time_taken (min)         45584 non-null  int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 3.5+ MB


In [15]:
df['Time_Taken (min)']=df['Time_taken (min)']
df.drop('Time_taken (min)',axis=1,inplace=True)
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_Taken (min)
0,36.0,4.2,Fog,Jam,2,motorcycle,3.0,No,Metropolitian,46
1,21.0,4.7,Stormy,High,1,motorcycle,1.0,No,Metropolitian,23
2,23.0,4.7,Sandstorms,Medium,1,scooter,1.0,No,Metropolitian,21
3,34.0,4.3,Sandstorms,Low,0,motorcycle,0.0,No,Metropolitian,20
4,24.0,4.7,Fog,Jam,1,scooter,1.0,No,Metropolitian,41


In [16]:
##independent and dependent features
X=df.drop(labels=['Time_Taken (min)'],axis=1)
y=df[['Time_Taken (min)']]

In [17]:
#Segregating numerical and categorical features and define which should be ordinal encodeed and which should be scaled
categorical_cols=X.select_dtypes(include='O').columns
numerical_cols=X.select_dtypes(exclude='O').columns


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Delivery_person_Age      43730 non-null  float64
 1   Delivery_person_Ratings  43676 non-null  float64
 2   Weather_conditions       44968 non-null  object 
 3   Road_traffic_density     44983 non-null  object 
 4   Vehicle_condition        45584 non-null  int64  
 5   Type_of_vehicle          45584 non-null  object 
 6   multiple_deliveries      44591 non-null  float64
 7   Festival                 45356 non-null  object 
 8   City                     44384 non-null  object 
 9   Time_Taken (min)         45584 non-null  int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 3.5+ MB


In [35]:
#Define the custom ranking for each ordinal features
Weather_conditions_cat=['Stormy','Sandstorms','Windy','Fog','Cloudy','Sunny']
Road_traffic_density_cat=['Low','Medium','High','Jam']
Type_of_vehicle_cat=['bicycle','electric_scooter','scooter','motorcycle']
Festival_cat=['No','Yes']
City_cat=['Semi-Urban','Urban','Metropolitian']
#category_cat=[Weather_conditions_cat,Road_traffic_density_cat,Type_of_vehicle_cat,Festival_cat,City_cat]

In [36]:
from sklearn.impute import SimpleImputer #for handling missing values
from sklearn.preprocessing import StandardScaler #for handling feature scaling
from sklearn.preprocessing import OrdinalEncoder #ordinal encoder

from sklearn.pipeline import Pipeline #for combining multiple steps
from sklearn.compose import ColumnTransformer#for connecting pipelines

In [37]:
##Numerical pipeline
num_pipeline=Pipeline(
    steps=[
    ('Imputer',SimpleImputer(strategy='median')),
    ('scalar',StandardScaler())
       ]
)
##categorical pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[Weather_conditions_cat,Road_traffic_density_cat,Type_of_vehicle_cat,Festival_cat,City_cat])),
    ('scalar',StandardScaler())
        ]
)
##combining cat and num pipelines
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [38]:
##Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [39]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [40]:
##Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [41]:
import numpy as np
def evaluate_model(true,predicted):
    mse=mean_squared_error(true,predicted)
    mae=mean_absolute_error(true,predicted)
    rmse=np.sqrt(mse)
    rsq=r2_score(true,predicted)
    return mse,mae,rmse,rsq

In [42]:
models={
    'Linear Regression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elastic Net':ElasticNet()
}

model_list=[]
rsq_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred=model.predict(X_test)
    mse,mae,rmse,rsq=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    print("-"*20)
    model_list.append(list(models.keys())[i])
    rsq_list.append(rsq*100)
    print('model training performance :')
    print('MSE :',mse)
    print('MAE :',mae)
    print('RMSE:',rmse)
    print('R-Sq:',rsq*100)
    print("="*30,'\n')

Linear Regression
--------------------
model training performance :
MSE : 46.053381452443205
MAE : 5.3597460333693725
RMSE: 6.786264174967196
R-Sq: 48.44672295117971

Lasso
--------------------
model training performance :
MSE : 51.45134486915368
MAE : 5.7416924671985745
RMSE: 7.172959282552333
R-Sq: 42.40411120922892

Ridge
--------------------
model training performance :
MSE : 46.05340265145377
MAE : 5.359750650540997
RMSE: 6.786265736872803
R-Sq: 48.44669922049087

Elastic Net
--------------------
model training performance :
MSE : 52.84023199106177
MAE : 5.841730509139369
RMSE: 7.269128695453244
R-Sq: 40.849357131958705



In [43]:
model_list

['Linear Regression', 'Lasso', 'Ridge', 'Elastic Net']

In [44]:
rsq_list

[48.44672295117971, 42.40411120922892, 48.44669922049087, 40.849357131958705]

In [45]:
print(f'The accuracy of the model {model_list[rsq_list.index(max(rsq_list))]} is higher with R-squared value {max(rsq_list)}')

The accuracy of the model Linear Regression is higher with R-squared value 48.44672295117971
